In [14]:
from cycling_manager.preprocess import get_data, preprocess, split
from cycling_manager.new_sequences import get_full_sequence, get_images

In [5]:
df = preprocess(get_data())

full df shape (603619, 25)
full df shape (603619, 25)
Score vertical meters - model: DecisionTreeRegressor() score: 0.9995672070999442
Score vertical meters - model: DecisionTreeRegressor() score: 0.9998823591752498
Score vertical meters - model: KNeighborsClassifier() score: 0.993762498316289


In [7]:
train, test = split(df, 2020, 2022)
train.head()

,name,year,race_name
472,magnus-cort-nielsen,2021,tour-de-france
634,maciej-bodnar,2021,giro-d-italia
622,jan-tratnik,2021,giro-d-italia
623,rafael-valls,2021,giro-d-italia
624,giovanni-visconti,2021,giro-d-italia


In [33]:
X_decoder_num, y_decoder, X_encoder_num_no_y, y_encoder_num, X_decoder_img_ls, X_encoder_img_ls, y_encoder_img_ls=\
    get_full_sequence(df, 'magnus-cort-nielsen', 2021, "tour-de-france", 40, 30, img=True, binary=True, y_encoder=True)

In [36]:
X_encoder_img_ls

['race/arctic-race-of-norway/2019/stage-3',
 'race/paris-nice/2021/stage-3',
 'race/paris-nice/2021/stage-5',
 'race/paris-nice/2021/stage-8',
 'race/itzulia-basque-country/2021/stage-5',
 'race/tour-de-romandie/2021/prologue',
 'race/tour-de-romandie/2021/stage-1',
 'race/tour-de-romandie/2021/stage-3',
 'race/tour-de-romandie/2021/stage-5',
 'race/la-route-d-occitanie/2021/stage-1']

In [35]:
X_encoder_img, X_decoder_img, y_encoder_img, to_drop_ls = get_images(X_decoder_img_ls, X_encoder_img_ls, y_encoder_img_ls, resnet=True)

['../raw_data/img_300/.jpg', '../raw_data/img_300/.jpg', '../raw_data/img_300/.jpg', '../raw_data/img_300/.jpg', '../raw_data/img_300/.jpg', '../raw_data/img_300/.jpg', '../raw_data/img_300/.jpg', '../raw_data/img_300/.jpg', '../raw_data/img_300/.jpg', '../raw_data/img_300/.jpg', '../raw_data/img_300/.jpg', '../raw_data/img_300/.jpg', '../raw_data/img_300/.jpg', '../raw_data/img_300/.jpg', '../raw_data/img_300/.jpg', '../raw_data/img_300/.jpg', '../raw_data/img_300/.jpg', '../raw_data/img_300/.jpg', '../raw_data/img_300/.jpg', '../raw_data/img_300/.jpg', '../raw_data/img_300/.jpg', '../raw_data/img_300/.jpg', '../raw_data/img_300/.jpg', '../raw_data/img_300/.jpg', '../raw_data/img_300/.jpg', '../raw_data/img_300/.jpg', '../raw_data/img_300/.jpg', '../raw_data/img_300/.jpg', '../raw_data/img_300/.jpg', '../raw_data/img_300/.jpg', '../raw_data/img_300/.jpg', '../raw_data/img_300/.jpg', '../raw_data/img_300/.jpg', '../raw_data/img_300/.jpg', '../raw_data/img_300/.jpg', '../raw_data/img_30

[ WARN:0@1662.437] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../raw_data/img_300/.jpg'): can't open/read file: check file path/integrity
[ WARN:0@1662.438] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../raw_data/img_300/.jpg'): can't open/read file: check file path/integrity
[ WARN:0@1662.438] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../raw_data/img_300/.jpg'): can't open/read file: check file path/integrity
[ WARN:0@1662.438] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../raw_data/img_300/.jpg'): can't open/read file: check file path/integrity
[ WARN:0@1662.438] global /Users/xperience/actions-r

UnboundLocalError: local variable 'tour_ls_img' referenced before assignment